# NLP Basic Assignment
## NLP 맛보기 - spam.csv를 가지고 유의미한 해석을 도출해주세요!

In [ ]:
# 필요한 모듈 불러오기
import pandas as pd

## Load Data
- 보시면 아시다시피 spam.csv는 라벨이 있는 데이터입니다. 물론 7주차 주제가 텍스트 기초인만큼 텍스트만 활용하셔도 되고, 라벨까지 활용하셔서 모델을 돌려보셔도 좋습니다 :)

In [ ]:
spam = pd.read_csv('spam.csv')

In [ ]:
spam.iloc[5]['v2']

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

In [ ]:
spam.info()
spam.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


(5572, 2)

In [ ]:
spam.dropna()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
spam.drop_duplicates(subset=['v2'], inplace=True)
spam.shape

(5169, 2)

## Tokenizing


토큰화

특수문자 제거

불용어 처리

소문자로 변경


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
word_tokenize(spam.iloc[5]['v2'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['FreeMsg',
 'Hey',
 'there',
 'darling',
 'it',
 "'s",
 'been',
 '3',
 'week',
 "'s",
 'now',
 'and',
 'no',
 'word',
 'back',
 '!',
 'I',
 "'d",
 'like',
 'some',
 'fun',
 'you',
 'up',
 'for',
 'it',
 'still',
 '?',
 'Tb',
 'ok',
 '!',
 'XxX',
 'std',
 'chgs',
 'to',
 'send',
 ',',
 'å£1.50',
 'to',
 'rcv']

In [ ]:
word = re.sub('[^a-zA-Z]', ' ', spam.iloc[5]['v2'])
words = word_tokenize(word)
print(words)

['FreeMsg', 'Hey', 'there', 'darling', 'it', 's', 'been', 'week', 's', 'now', 'and', 'no', 'word', 'back', 'I', 'd', 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', 'Tb', 'ok', 'XxX', 'std', 'chgs', 'to', 'send', 'to', 'rcv']


In [ ]:
nltk.download('stopwords')
stops = set(stopwords.words('english'))
words = [word for word in words if word not in stops]
print(words)

['FreeMsg', 'Hey', 'darling', 'week', 'word', 'back', 'I', 'like', 'fun', 'still', 'Tb', 'ok', 'XxX', 'std', 'chgs', 'send', 'rcv']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
words = [word.lower() for word in words]
print(words)

['freemsg', 'hey', 'darling', 'week', 'word', 'back', 'i', 'like', 'fun', 'still', 'tb', 'ok', 'xxx', 'std', 'chgs', 'send', 'rcv']


In [ ]:
spam.iloc[5]['v2']

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

In [ ]:
spam_token = []
for i in range(len(spam)):
    words = re.sub('[^a-zA-Z]', ' ', spam.iloc[i]['v2'])  
    words = word_tokenize(words) 
    words = [word for word in words if word not in stops]  
    words = [word.lower() for word in words]  
    spam_token.append(words)

## Embedding

- 수업에서 다룬 임베딩 방법에는 One-hot encoding, CBOW, Skip-gram, GloVe, FastText가 있었습니다. 다양한 시도와 '비교' 결과를 함께 적어주세요! 파라미터를 조정해가는 과정도 해석에 도움이 될 수 있겠죠 :)

In [ ]:
from gensim.models import Word2Vec

CBOW = Word2Vec(spam_token, size=2, window=2, min_count=1, workers=1, iter=1000, sg=0)
CBOW_words = CBOW.wv.index2word
CBOW_vectors = CBOW.wv.vectors

In [ ]:
CBOW_df = pd.DataFrame(CBOW_vectors, columns = ['x1', 'x2'])
CBOW_df['words'] = CBOW_words
CBOW_df = CBOW_df[['words', 'x1', 'x2']]
CBOW_df

,words,x1,x2
0,i,-0.200380,1.608772
1,u,0.107427,0.971029
2,call,1.872313,-0.627883
3,get,0.781537,0.218207
4,ur,0.993146,-0.066321
...,...,...,...
7678,lowes,-1.712537,3.934646
7679,salesman,-0.924200,1.962608
7680,pity,-0.728874,1.990397
7681,suggestions,-0.374247,2.783558


In [ ]:
Skip_Gram = Word2Vec(spam_token, size=2, window=3, min_count=1, workers=1, iter=1000, sg=1)
Skip_Gram_words = Skip_Gram.wv.index2word
Skip_Gram_vectors = Skip_Gram.wv.vectors

Skip_Gram_df = pd.DataFrame(Skip_Gram_vectors, columns = ['x1', 'x2'])
Skip_Gram_df['words'] = Skip_Gram_words
Skip_Gram_df = Skip_Gram_df[['words', 'x1', 'x2']]

In [ ]:
from gensim.models import FastText

Fastext = FastText(spam_token, size=100, window=5, min_count=5, workers=4, sg=1)
Fastext_words = Fastext.wv.index2word
Fastext_vectors = Fastext.wv.vectors

CBOW,Skip_gram,Fastext 3가지방식으로 워드임베딩을 진행하였다.

## 본인이 도출해낸 해석을 적어주세요!

- 유사도, Wordcloud, 이진 분류 모델, Plot 뭐든 상관없으니 분명하고 인상적인 해석을 적어주시면 됩니다.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

ham_index = spam.loc[spam.v1=='ham',].index

ham_oneline = ""
for i in ham_index:
    for j in spam_token[i]:
        ham_oneline = ham_oneline + j + " "

wc = WordCloud(width=1000, height=600, background_color="white", random_state=0)
plt.imshow(wc.generate(ham_oneline))
plt.axis("off")
plt.show()

In [ ]:
spam_index = spam.loc[spam.v1=='spam',].index

spam_oneline = ""
for i in spam_index:
    for j in spam_token[i]:
        spam_oneline = spam_oneline + j + " "

wc = WordCloud(width=1000, height=600, background_color="white", random_state=0)
plt.imshow(wc.generate(spam_oneline))
plt.axis("off")
plt.show()

Wordcloud를 그려보니 spam_data에서 free,text,call,txt등이 많이 나옴을 알수 있다. 이는 실제 스팸메일에 있을법한 단어이고 해당 모델이 스팸메일과 스팸메일이 아닌 문장을 잘 구분했음을 확인할수있다. 